# **Assignment 11**

## **Part 1**

### **Load the data**


In [5]:
import pandas as pd

df = pd.read_parquet('./teachers_db_practice.parquet')

### **Data processing**

In [6]:
df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   area         1228 non-null   object 
 1   area_code    1228 non-null   int64  
 2   position     274 non-null    object 
 3   full_info    1228 non-null   object 
 4   id           1219 non-null   float64
 5   description  1228 non-null   object 
 6   tags         1204 non-null   object 
 7   published    1228 non-null   object 
 8   size         1219 non-null   float64
 9   width        1219 non-null   float64
 10  height       1219 non-null   float64
 11  orientation  1228 non-null   object 
 12  site         1228 non-null   object 
 13  gender       1228 non-null   int64  
 14  alias        1228 non-null   object 
dtypes: float64(4), int64(2), object(9)
memory usage: 144.0+ KB


,area,area_code,position,full_info,id,description,tags,published,size,width,height,orientation,site,gender,alias
0,Architecture & Design,40358,None,<p> has worked as a designer for the last dec...,184670.0,,[],2025-06-16T11:48:18.952Z,22834.0,500.0,500.0,S,global,0,Appius Aemilius Agricola
1,Economics,17166,None,<p>Mr. Madgar has been teaching economics par...,16106.0,Close-up portrait of a smiling man with a bear...,"[man, portrait, smiling, beard, close-up]",2022-11-18T07:21:20.407Z,7180.0,170.0,170.0,S,global,0,Appius Aemilius Cicero
2,Private & Business Law,40353,None,<p>Lawyer with broad experience in Market Regu...,25635.0,,[],2023-03-17T12:44:46.932Z,14699.0,128.0,128.0,S,global,0,Appius Aemilius Crassus
3,Economics,17166,Adjunct professor,<p> is a seasoned leader with a proven track r...,37534.0,A professional portrait of a smiling woman wit...,"[woman, portrait, smiling, blond hair, blue bl...",2024-02-15T12:47:08.017Z,79371.0,500.0,500.0,S,global,1,Flavia Prisca
4,Science & Technology,40359,None,"<p> Carrio is a seasoned technology leader, re...",182500.0,,[],2025-04-07T10:15:21.908Z,15906.0,500.0,500.0,S,global,0,Appius Aemilius Scipio


In [7]:
df_professors = df[['alias', 'area', 'full_info']].copy()

display(df_professors.head())
display(df_professors.info())

,alias,area,full_info
0,Appius Aemilius Agricola,Architecture & Design,<p> has worked as a designer for the last dec...
1,Appius Aemilius Cicero,Economics,<p>Mr. Madgar has been teaching economics par...
2,Appius Aemilius Crassus,Private & Business Law,<p>Lawyer with broad experience in Market Regu...
3,Flavia Prisca,Economics,<p> is a seasoned leader with a proven track r...
4,Appius Aemilius Scipio,Science & Technology,"<p> Carrio is a seasoned technology leader, re..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   alias      1228 non-null   object
 1   area       1228 non-null   object
 2   full_info  1228 non-null   object
dtypes: object(3)
memory usage: 28.9+ KB


None

### **Named Entity Recognition:**

 ### **Imports, Model Setup & Pattern**s

In [10]:
!pip3 install networkx

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [11]:
!pip3 install pandas transformers torch beautifulsoup4 lxml networkx rapidfuzz
import re
import html
import json
import unicodedata
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional, Tuple

import pandas as pd
import networkx as nx
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline



MODEL_ID = "dslim/bert-base-NER"
tok = AutoTokenizer.from_pretrained(MODEL_ID)
ner_model = AutoModelForTokenClassification.from_pretrained(MODEL_ID)
ner = pipeline("ner", model=ner_model, tokenizer=tok, aggregation_strategy="simple")

YEAR_PAT = re.compile(r"\b(19|20)\d{2}\b")
RANGE_PAT = re.compile(r"\b(?P<start>(?:19|20)\d{2})\s*[\u2012\u2013\u2014\u2015\-–—]\s*(?P<end>(?:Present|present|(?:19|20)\d{2}))\b")
DEGREE_PAT = re.compile(
    r"\b(Ph\.?D\.?|Doctor(?:ate)?|M\.?Sc\.?|MSc|M\.?A\.?|MA|MBA|E\.?M\.?B\.?A\.?|Executive MBA|LL\.?M\.?|B\.?Sc\.?|BSc|B\.?A\.?|BA|Bachelor|Master['’]s?)\b",
    re.I,
)
COURSE_KEYWORDS = re.compile(r"\b(Certificate|Program(?:me)?|Course|Diploma|Seminar|Workshop)\b", re.I)
ROLE_SPLIT_COMMAS = re.compile(r"\s*,\s*")
HEADING_ALIASES = {
    "corporate": re.compile(r"^\s*Corporate Experience\s*$", re.I),
    "academic_exp": re.compile(r"^\s*Academic Experience\s*$", re.I),
    "academic_bg": re.compile(r"^\s*Academic Background\s*$", re.I),
    "corpcaps": re.compile(r"^\s*CORPORATE EXPERIENCE\s*$", re.I),
    "acadcaps": re.compile(r"^\s*ACADEMIC EXPERIENCE\s*$", re.I),
    "acadbgcaps": re.compile(r"^\s*ACADEMIC BACKGROUND\s*$", re.I),
}


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

ModuleNotFoundError: No module named 'networkx'

In this cell, we’ve set up all the main imports, regex patterns, and the NER model. We decided to use the “dslim/bert-base-NER” model from Hugging Face because it’s lightweight, fast, and works really well with English text. It can easily recognize things like organizations, locations, and people, which is perfect for identifying universities, companies, and academic institutions in the professors’ biographies.

### **Set Up Extraction Logic**

In this part, we define several helper functions that clean and parse the professors’ biographies.
The goal is to turn each HTML biography (full_info) into structured data like universities, degrees, or companies that we can later use to build the knowledge graph.
Each helper function has its own role:

- normalize_text → cleans up HTML and punctuation

- html_to_lines → breaks the bio into clear text lines

- detect_sections → separates parts like “Corporate” or “Academic Background”

- extract_* functions → pull out details such as degree, institution, year, or company

- extract_professor_info → ties everything together and outputs a dictionary with structured entities.

In [ ]:
from typing import Optional, List, Dict, Any

# Set up Helper functions required by extract_professor_info()

import re
import html
from bs4 import BeautifulSoup

# --- normalize text, remove html noise ---
def normalize_text(s):
    """Clean up whitespace, HTML entities, and punctuation for consistent parsing."""
    if not s:
        return s
    s = html.unescape(s)
    s = s.replace("\xa0", " ").strip(" ,;\t\n\r")
    return s

# --- convert html into a list of text lines ---
def html_to_lines(raw):
    """Turn professor HTML blob into clean, bullet-style lines for easier parsing."""
    soup = BeautifulSoup(raw, "lxml")
    for br in soup.find_all("br"):
        br.replace_with("\n")

    lines = []
    for el in soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p", "li"]):
        txt = el.get_text(" ", strip=True)
        txt = normalize_text(txt)
        if txt:
            bullets = [t.strip() for t in txt.split("•") if t.strip()]
            if len(bullets) > 1:
                lines.extend([f"• {b}" for b in bullets])
            else:
                lines.append(txt)

    if not lines:
        txt = normalize_text(re.sub(r"<[^>]+>", " ", raw))
        lines = [t.strip() for t in re.split(r"\n+|\r+", txt) if t.strip()]
    return lines

# --- detect section headings to split biography text logically ---
def detect_sections(lines):
    """Separates text into logical sections using headings like 'Corporate Experience' or 'Academic Background'."""
    sections = {"lead": [], "Corporate Experience": [], "Academic Experience": [], "Academic Background": []}
    current = "lead"

    for line in lines:
        if re.search(r"Corporate Experience", line, re.I):
            current = "Corporate Experience"
            continue
        if re.search(r"Academic Experience", line, re.I):
            current = "Academic Experience"
            continue
        if re.search(r"Academic Background", line, re.I):
            current = "Academic Background"
            continue
        sections[current].append(line)

    return sections

# --- run NER model ---
def run_ner(text):
    """Run NER model on text to detect organizations and locations."""
    if not text.strip():
        return []
    return ner(text)

# --- extract role/company/location/year info from a corporate line ---
def extract_role_company_loc_years(line, ents):
    """Extracts role, company, location, and years from corporate experience lines."""
    parts = [p.strip() for p in line.lstrip("• ").split(",")]
    role = parts[0] if len(parts) > 0 else None
    company = parts[1] if len(parts) > 1 else None
    location = parts[2] if len(parts) > 2 else None

    # look for years like '2017 – 2023'
    match = re.search(r"(19|20)\d{2}\s*[–-]\s*((19|20)\d{2}|Present)", line)
    if match:
        date_start, date_end = match.group(0).split("–") if "–" in match.group(0) else match.group(0).split("-")
        date_start, date_end = date_start.strip(), date_end.strip()
    else:
        date_start = date_end = None

    return {"role": role, "name": company, "location": location, "date_start": date_start, "date_end": date_end}

# --- extract degree, field, institution, etc. from academic background line ---
def extract_study_line(line, ents):
    """Extracts degree, field, institution, location, and year from an academic background line."""
    degree_match = re.search(r"(Bachelor|Master|Ph\.?D\.?|MBA|LL\.?M\.?)", line, re.I)
    degree = degree_match.group(0) if degree_match else None

    field_match = re.search(r"in\s+([^,]+)", line, re.I)
    field = field_match.group(1).strip() if field_match else None

    inst_match = re.search(r",\s*([^,]+University[^,]*|[^,]+College[^,]*|[^,]+School[^,]*)", line)
    institution = inst_match.group(1).strip() if inst_match else None

    loc_match = re.search(r"(USA|UK|Spain|Mexico|Belgium|France|Germany|Italy|U\.K\.|U\.S\.A\.)", line)
    location = loc_match.group(0) if loc_match else None

    year_match = re.search(r"\b(19|20)\d{2}\b", line)
    year = year_match.group(0) if year_match else None

    is_course = bool(re.search(r"(Certificate|Course|Program|Diploma|Seminar|Workshop)", line, re.I))

    return {
        "degree": degree,
        "field": field,
        "institution": institution,
        "location": location,
        "year": year,
        "is_course": is_course,
        "raw": line
    }


# --- helper to map course keyword to a 'program' label ---
def _course_program_from_text(text: str) -> Optional[str]:  # maps Certificate/Program/Course/... to program label
    m = COURSE_KEYWORDS.search(text)
    if not m:
        return None
    kw = m.group(1).lower()
    if "certificate" in kw: return "Certificate"
    if "programme" in kw or "program" in kw: return "Program"
    if "course" in kw: return "Course"
    if "diploma" in kw: return "Diploma"
    if "seminar" in kw: return "Seminar"
    if "workshop" in kw: return "Workshop"
    return None



In [ ]:
# Main extraction function

def extract_professor_info(html_blob: str) -> Dict[str, List[Dict[str, Any]]]:
    """
    Extracts structured information about one professor from the HTML text.
    Returns a dictionary with four structured lists:
      - universities: [{name, location, degree, field, year}]
      - studies:      [{degree, field, year, institution, location}]
      - companies:    [{name, location, role, date_start, date_end}]
      - courses:      [{subject, program, institution, year}]
    """

    # --- Step 1: Clean and section the HTML ---
    lines = html_to_lines(html_blob)              # convert HTML into clean bullet/paragraph lines
    sections = detect_sections(lines)             # split text into logical parts (Corporate / Academic)
    ents = run_ner("\n".join(lines))              # run NER once for efficiency

    # --- Step 2: Initialize output lists ---
    universities: List[Dict[str, Any]] = []
    studies: List[Dict[str, Any]] = []
    companies: List[Dict[str, Any]] = []
    courses: List[Dict[str, Any]] = []

    # --- Step 3: Corporate Experience (Companies) ---
    for line in sections.get("Corporate Experience", []):
        if not line.strip().startswith("•"):
            continue
        c = extract_role_company_loc_years(line, ents)
        if c.get("name") or c.get("role"):
            companies.append({
                "name": c.get("name"),
                "location": c.get("location"),
                "role": c.get("role"),
                "date_start": c.get("date_start"),
                "date_end": c.get("date_end")
            })

    # --- Step 4: Academic Background (Universities, Studies, Courses) ---
    for line in sections.get("Academic Background", []):
        if not line.strip().startswith("•"):
            continue
        s = extract_study_line(line, ents)
        if s["is_course"]:
            # Extract course subject and normalize program type
            mck = COURSE_KEYWORDS.search(s["raw"])
            subject = normalize_text(s["raw"][mck.start():]) if mck else normalize_text(s["raw"])
            program = _course_program_from_text(s["raw"])
            courses.append({
                "subject": subject,
                "program": program,
                "institution": s["institution"],
                "year": s["year"]
            })
        else:
            # Add Study entry
            studies.append({
                "degree": s["degree"],
                "field": s["field"],
                "year": s["year"],
                "institution": s["institution"],
                "location": s["location"]
            })
            # Add University entry
            if s["institution"]:
                universities.append({
                    "name": s["institution"],
                    "location": s["location"],
                    "degree": s["degree"],
                    "field": s["field"],
                    "year": s["year"]
                })

    # --- Step 5: Academic Experience (teaching roles treated as Companies) ---
    for line in sections.get("Academic Experience", []):
        if not line.strip().startswith("•"):
            continue
        c = extract_role_company_loc_years(line, ents)
        if c.get("name") or c.get("role"):
            companies.append({
                "name": c.get("name"),
                "location": c.get("location"),
                "role": c.get("role"),
                "date_start": c.get("date_start"),
                "date_end": c.get("date_end")
            })

    # --- Step 6: Deduplicate by meaningful keys ---
    def dedup(records, keyfunc):
        seen, out = set(), []
        for r in records:
            k = keyfunc(r)
            if k not in seen:
                out.append(r)
                seen.add(k)
        return out

    universities = dedup(universities, lambda x: (x["name"], x["degree"], x["field"], x["year"], x["location"]))
    studies      = dedup(studies,      lambda x: (x["institution"], x["degree"], x["field"], x["year"], x["location"]))
    companies    = dedup(companies,    lambda x: (x["name"], x["role"], x["date_start"], x["date_end"], x["location"]))
    courses      = dedup(courses,      lambda x: (x["subject"], x["program"], x["institution"], x["year"]))

    # --- Step 7: Return fully structured data ---
    return {
        "universities": universities,
        "studies": studies,
        "companies": companies,
        "courses": courses
    }

### **Apply extraction logic to df_professors**

Here we apply the extract_professor_info() function to every professor’s biography in the dataset.
This step turns the raw text inside full_info into structured entities  (universities, studies, companies, and courses) for each person.
Then, we flatten those nested results into separate tables (df_universities, df_studies, df_companies, df_courses) so we can easily explore relationships between professors, institutions, and industries later when building the knowledge graph.

In [ ]:
# ======================================
# Apply extraction to df_professors
# ======================================

# Apply the extraction function to each professor’s 'full_info' field
df_professors["extracted"] = df_professors["full_info"].apply(extract_professor_info)

# --- Flatten the extracted data into separate normalized tables ---

rows_univ = []
rows_study = []
rows_company = []
rows_course = []

for idx, row in df_professors.iterrows():
    alias = row["alias"]
    area = row["area"]
    extracted = row["extracted"]

    # Add a reference to the professor (alias & area) in each record
    for u in extracted["universities"]:
        rows_univ.append({"alias": alias, "area": area, **u})
    for s in extracted["studies"]:
        rows_study.append({"alias": alias, "area": area, **s})
    for c in extracted["companies"]:
        rows_company.append({"alias": alias, "area": area, **c})
    for c in extracted["courses"]:
        rows_course.append({"alias": alias, "area": area, **c})

# Create normalized DataFrames for each entity type
df_universities = pd.DataFrame(rows_univ)
df_studies = pd.DataFrame(rows_study)
df_companies = pd.DataFrame(rows_company)
df_courses = pd.DataFrame(rows_course)

# 3. Cleanup / normalization
def _to_year(x):
    if not x or str(x).strip().lower() in ("", "none", "present"): return None
    try: return int(x)
    except: return None

def _clean_none(x):
    if x in (None, "", "None", "/", "/ USA"): return None
    return x

def _normalize_present(x):
    if x is None: return None
    return None if str(x).strip().lower() == "present" else x

def _explode_slash(df, col):
    if col not in df.columns: return df
    tmp = df.copy()
    tmp[col] = tmp[col].apply(lambda v: [v] if (not isinstance(v, str) or "/" not in v)
                              else [p.strip() for p in v.split("/") if p.strip()])
    return tmp.explode(col, ignore_index=True)

# Clean tables
for dfx in (df_universities, df_studies):
    dfx["year"] = dfx["year"].apply(_to_year)
    for c in ("degree","field","location","institution","name"):
        if c in dfx.columns:
            dfx[c] = dfx[c].apply(lambda z: z.strip() if isinstance(z,str) else z)
            dfx[c] = dfx[c].apply(_clean_none)

df_companies["date_start"] = df_companies["date_start"].apply(_to_year)
df_companies["date_end"]   = df_companies["date_end"].apply(_normalize_present)
for c in ("name","role","location"):
    df_companies[c] = df_companies[c].apply(lambda z: z.strip() if isinstance(z,str) else z)

df_universities = _explode_slash(df_universities, "name")
df_studies      = _explode_slash(df_studies,      "institution")

def _drop_all_null(df, cols): return df.dropna(subset=cols, how="all")
df_universities = _drop_all_null(df_universities, ["name","degree","field","year","location"]).drop_duplicates()
df_studies      = _drop_all_null(df_studies,      ["institution","degree","field","year","location"]).drop_duplicates()
df_companies    = _drop_all_null(df_companies,    ["name","role","date_start","date_end","location"]).drop_duplicates()
df_courses      = _drop_all_null(df_courses,      ["subject","program","institution","year"]).drop_duplicates()

# 4. Display cleaned samples
display(df_universities.head(10))
display(df_studies.head(10))
display(df_companies.head(10))
display(df_courses.head(10))


These four tables show the structured information automatically extracted from each professor’s biography.

1. Universities / Studies: list academic institutions, degrees, fields, and years. The results show the model correctly separated composite entries like “IE Business School / Brown University.”

2. Companies: summarize professional experience, including role, company, location, and time range. The date normalization removed “Present” and standardized years.

3. Courses: contain additional training such as certificates, seminars, or programs.

Overall, the extraction pipeline successfully turned unstructured biographies into clean tabular data, which can now be used to build the knowledge graph.

### **Sketch Knowledge Graph**

First of all, since GraphML doesn’t accept missing values (None, NaN) or complex Python objects we create this helper function to clean the graph’s node and edge attributes before saving it as a GraphML file. This step ensures the following:

- Empty or invalid values are removed.

- Only simple types (strings, numbers, booleans) are kept.

- More complex values (like lists or dicts) are converted to JSON strings.

In [ ]:
# Preparation: Create function Scrub Data (drop emty values)

import math
import json
import networkx as nx
import pandas as pd

def _to_graphml_value(v):
    # Drop Nones / NaNs
    if v is None:
        return None
    if isinstance(v, float) and math.isnan(v):
        return None
    # Allowed primitives
    if isinstance(v, (str, int, float, bool)):
        return v
    # Everything else: JSON-stringify
    return json.dumps(v, ensure_ascii=False)

def scrub_graphml_inplace(G: nx.MultiDiGraph) -> None:
    # Clean node attrs
    for n, data in G.nodes(data=True):
        for k in list(data.keys()):
            v = _to_graphml_value(data[k])
            if v is None:
                del data[k]
            else:
                data[k] = v
    # Clean edge attrs
    for u, v, k, data in G.edges(keys=True, data=True):
        for attr in list(data.keys()):
            val = _to_graphml_value(data[attr])
            if val is None:
                del data[attr]
            else:
                data[attr] = val


In this next cell, we will use the cleaned and normalized tables (professors, universities, companies, and courses) to construct a directed multi-type knowledge graph with NetworkX.
Each node represents either a professor (blue), a company/organization (green), a university (purple), or an institution providing courses (orange).
The edges capture three types of relationships:

1. worked_at → connects professors to organizations they’ve worked for, including their role, start/end years, and location.

2. studied_at → connects professors to universities, with attributes such as degree, field, and year.

3. took_course → connects professors to institutions where they completed certificates or training programs.

The function build_kg_from_tables() organizes all this information into a MultiDiGraph, meaning multiple types of edges can exist between the same pair of nodes.

After building the graph, we clean it using scrub_graphml_inplace() to remove any invalid or empty attributes before exporting it to a .graphml file, making it compatible with visualization tools like Gephi.

In [ ]:
# ======================================
# Knowledge Graph (KG) sketch with NX
# ======================================

import networkx as nx
import pandas as pd

def build_kg_from_tables(
    df_professors: pd.DataFrame,
    df_universities: pd.DataFrame,
    df_studies: pd.DataFrame,
    df_companies: pd.DataFrame,
    df_courses: pd.DataFrame,
) -> nx.MultiDiGraph:
    """
    Build a MultiDiGraph where:
      - Nodes:
          ("professor", alias)        with attrs: label, area
          ("organization", name)      with attrs: label, location
          ("university", name)        with attrs: label, location
          ("institution", name)       with attrs: label, location (optional)
      - Edges (directed, professor -> org):
          worked_at   (role, start, end, location)
          studied_at  (degree, field, year, location)
          took_course (course, program, year)
    """
    G = nx.MultiDiGraph()

    # --- Professors as nodes (alias = stable id) ---
    for _, r in df_professors.iterrows():
        alias = r["alias"]
        G.add_node(("professor", alias), label=alias, type="professor", area=r.get("area"))

    # helper to add/get org-like nodes with optional location merging
    def add_org_node(name: str, node_type: str, location: str | None = None):
        if not name:
            return None
        nid = (node_type, name)
        if not G.has_node(nid):
            G.add_node(nid, label=name, type=node_type, location=location)
        else:
            if location and not G.nodes[nid].get("location"):
                G.nodes[nid]["location"] = location
        return nid

    # --- Companies -> worked_at edges ---
    # expected columns: alias, name, location, role, date_start, date_end
    if not df_companies.empty:
        for _, r in df_companies.iterrows():
            src = ("professor", r["alias"])
            tgt = add_org_node(r.get("name"), "organization", r.get("location"))
            if tgt:
                G.add_edge(
                    src, tgt,
                    relation="worked_at",
                    role=r.get("role"),
                    start=r.get("date_start"),
                    end=r.get("date_end"),
                    location=r.get("location"),
                )

    # --- Universities -> studied_at edges ---
    # expected columns: alias, name, location, degree, field, year
    if not df_universities.empty:
        for _, r in df_universities.iterrows():
            src = ("professor", r["alias"])
            tgt = add_org_node(r.get("name"), "university", r.get("location"))
            if tgt:
                G.add_edge(
                    src, tgt,
                    relation="studied_at",
                    degree=r.get("degree"),
                    field=r.get("field"),
                    year=r.get("year"),
                    location=r.get("location"),
                )

    # --- Courses -> took_course edges ---
    # expected columns: alias, subject, program, institution, year
    if not df_courses.empty:
        for _, r in df_courses.iterrows():
            src = ("professor", r["alias"])
            inst_name = r.get("institution") or "Unknown Institution"
            tgt = add_org_node(inst_name, "institution")  # course providers as 'institution'
            if tgt:
                G.add_edge(
                    src, tgt,
                    relation="took_course",
                    course=r.get("subject"),
                    program=r.get("program"),
                    year=r.get("year"),
                )

    return G


# =========================
# Build + inspect the graph
# =========================

G = build_kg_from_tables(df_professors, df_universities, df_studies, df_companies, df_courses)
# scrub data (opr empty values)
scrub_graphml_inplace(G)
nx.write_graphml(G, "professors_kg.graphml")

# Optional: quick edge triplets preview
triplets = []
for u, v, d in G.edges(data=True):
    triplets.append({
        "source": G.nodes[u]["label"],
        "relation": d.get("relation"),
        "target": G.nodes[v]["label"],
        **{k: v for k, v in d.items() if k != "relation"}
    })
df_triplets = pd.DataFrame(triplets).sort_values(["source", "relation", "target"])


In [ ]:
# ============================
# Static KG sketch (matplotlib)
# ============================
import matplotlib.pyplot as plt
import networkx as nx

def visualize_kg_sketch(G: nx.MultiDiGraph, focus_alias: str | None = None, hops: int = 1, figsize=(12, 9)):
    """
    Draw a simple sketch of the KG.
    - focus_alias: if set, show only that professor and nodes within N hops.
    - hops: 1 = direct neighbors, 2 = neighbors-of-neighbors, etc.
    """
    # pick subgraph
    if focus_alias:
        center = ("professor", focus_alias)
        if center not in G:
            raise ValueError(f"Professor alias '{focus_alias}' not found in graph.")
        nodes = {center}
        frontier = {center}
        for _ in range(hops):
            next_frontier = set()
            for n in frontier:
                next_frontier.update(G.predecessors(n))
                next_frontier.update(G.successors(n))
            nodes.update(next_frontier)
            frontier = next_frontier
        H = G.subgraph(nodes).copy()
    else:
        # if whole graph is huge, sample a small induced subgraph (first 150 nodes)
        H = G.copy()
        if H.number_of_nodes() > 150:
            keep = list(H.nodes())[:150]
            H = H.subgraph(keep).copy()

    # colors by type
    type_color = {
        "professor": "#1f77b4",   # blue
        "organization": "#2ca02c",# green
        "university": "#9467bd",  # purple
        "institution": "#ff7f0e", # orange
    }
    node_colors = []
    node_labels = {}
    for n, data in H.nodes(data=True):
        t = data.get("type", "organization")
        node_colors.append(type_color.get(t, "#7f7f7f"))
        node_labels[n] = data.get("label", str(n))

    # layout
    pos = nx.spring_layout(H, k=0.7 / (1 + H.number_of_nodes()**0.5), seed=42)

    plt.figure(figsize=figsize)
    nx.draw_networkx_nodes(H, pos, node_color=node_colors, node_size=700, linewidths=0.8, edgecolors="white")
    nx.draw_networkx_edges(H, pos, width=1.0, alpha=0.6, arrows=True, arrowsize=12, connectionstyle="arc3,rad=0.08")
    nx.draw_networkx_labels(H, pos, labels=node_labels, font_size=8)

    # edge labels = relation
    edge_labels = {(u, v): d.get("relation", "") for u, v, d in H.edges(data=True)}
    nx.draw_networkx_edge_labels(H, pos, edge_labels=edge_labels, font_size=7, label_pos=0.5)

    plt.axis("off")
    title = f"Knowledge Graph Sketch ({'focus: ' + focus_alias if focus_alias else 'sample'})"
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Example usage:
visualize_kg_sketch(G)                                  # sample of whole graph
visualize_kg_sketch(G, focus_alias="Aemilia Ahenobarba") # focus on one professor


The first graph (sample view) shows the overall scale of the network. Most nodes are professors, so the graph looks circular with few visible links — it mainly reflects the dataset’s size and diversity.

The second graph (focus view) centers on Aemilia Ahenobarba, clearly showing her academic and professional connections. Blue represents her, while green and purple nodes show the organizations and universities she’s linked to. This view confirms that the graph correctly connects professors to their education and work history, showing meaningful relationships extracted from the data.

## **Part 2: LLM Grader Chatbot**

Functionalities required for streamlit app:

1.   Loads questions and target (“standard”) answers from Q&A_db_practice.json.
2.   Shows one question at a time.
3.	Lets the user (student) type an answer.
4.	Scores it by comparing it to the reference answer using text similarity metrics (e.g., ROUGE-like F1, TF-IDF cosine).
5.	Explains differences using either rule-based text comparison or a small LLM model.
6.	Optionally analyzes user feedback sentiment (via VADER).
7.	Logs results and allows CSV download.

### **Approach followed and rationale**

To successfully design an automated system that can evaluate responses to open-ended questions in ML we created a streamlit-based chatbot that compares answers to a curated dataset of “standard” answers from the *Q&A_db_practice.json* dataset. Instead of relying on deep learning models or external APIs, we used deterministic text similarity metrics for transparent evaluation. Specifically, three complementary methods were combined:
1. Token F1 overlap (to capture exact keyword matching)
2. ROUGE-L F1 (to measure shared sequence structure and phrasing)
3. TF-IDF cosine similarity (to detect overall semantic similarity)

Each metric contributes to a weighted composite score ranging from 0 to 100, giving us a balanced assessment between lexical precision and contextual relevance. Also, we used a rule-based feedback function that highlights missing key terms and phrasing differences to justify why answers received a particular score. We also included sentiment analysis using the VADER lexicon, enabling qualitative feedback assessment, and stores all results for later analysis.

This approach prioritizes interpretability and reliability which is important for an academic grading assistant that must run consistently without GPU dependencies. However, it does not capture deeper semantic relationships or rephrased explanations as effectively as transformer-based models. To improve this, we could fine-tune or integrate a LLM such as FLAN-T5, BERTScore, or GPT-style evaluation frameworks to get contextual scoring and natural language feedback. Another improvement would be supervised learning on annotated student responses, where a regression or classification model learns to predict human-assigned scores. Also, methods like embedding-based similarity (e.g., Sentence-BERT) or semantic entailment detection could make scoring more reasonable by understanding meaning rather than surface-level similarity. To successfully do this we would need labeled data, more computational resources, and robust validation to ensure fairness and explainability.

In [ ]:
# cleaning servers when necessary
!pkill -f streamlit || true
!pkill -f cloudflared || true
!pip -q install -U streamlit transformers scikit-learn nltk rapidfuzz torch
!python -m nltk.downloader vader_lexicon

In [ ]:
!pip -q install -U streamlit transformers scikit-learn nltk rapidfuzz torch
!python -m nltk.downloader vader_lexicon

In [ ]:
!pip -q install -U sentencepiece accelerate protobuf<5

In [ ]:
import pandas as pd

df = pd.read_json('/content/Q&A_db_practice.json')
df.head()

Call streamlit app from this file 

In [ ]:
import subprocess, shlex, time, re, pathlib

# install cloudflared once
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null

# start streamlit and tunneln
subprocess.Popen(shlex.split("streamlit run app.py --server.port 8501 --server.headless true"),
                 stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
log_path = "/content/cf.log"
subprocess.Popen(shlex.split("cloudflared tunnel --url http://localhost:8501 --no-autoupdate"),
                 stdout=open(log_path,"w"), stderr=subprocess.STDOUT)
time.sleep(5)
txt = pathlib.Path(log_path).read_text()
m = re.search(r"https://[\w-]+\.trycloudflare\.com", txt)
print("PUBLIC URL:", m.group(0) if m else "still starting… run again in 5s")